In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip3 install sklearn

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from sklearn.preprocessing import RobustScaler,StandardScaler
rb = RobustScaler()
sc = StandardScaler()